Почти весь код позаимствован с https://medium.com/@r3d_robot/getting-contextualized-word-embeddings-with-bert-20798d8b43a4.

# Contextual BERT embeddings

# 1. Setup

In [10]:
import pandas as pd
import numpy as np
import torch

In [11]:
!pip install transformers

In [12]:
from transformers import BertModel, BertTokenizer

model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased',
           output_hidden_states = True,)
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

# 2. Create contextual embeddings

We have to put the input text into a specific format that BERT can read. Mainly we add the ```[CLS]``` to the beginning and ```[SEP]``` to the end of the input. Then we convert the tokenized BERT input to the tensor format.

In [13]:
def bert_text_preparation(text, tokenizer):
  """
  Preprocesses text input in a way that BERT can interpret.
  """
  marked_text = "[CLS] " + text + " [SEP]"
  tokenized_text = tokenizer.tokenize(marked_text)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  segments_ids = [1]*len(indexed_tokens)

  # convert inputs to tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensor = torch.tensor([segments_ids])

  return tokenized_text, tokens_tensor, segments_tensor

In order to obtain the actual BERT embeddings, we take preprocessed input text, which now is represented by tensors, put it into our pre-trained BERT model.

Which vector works best as a contextualized embedding? I would think it depends on the task. The original paper that proposed BERT examines six choices.

I go with one of these choice that worked well in their experiments, which is the sum of the last four layers of the model.

In [14]:
def get_bert_embeddings(tokens_tensor, segments_tensor, model):
    """
    Obtains BERT embeddings for tokens, in context of the given sentence.
    """
    # gradient calculation id disabled
    with torch.no_grad():
      # obtain hidden states
      outputs = model(tokens_tensor, segments_tensor)
      hidden_states = outputs[2]

    # concatenate the tensors for all layers
    # use "stack" to create new dimension in tensor
    token_embeddings = torch.stack(hidden_states, dim=0)

    # remove dimension 1, the "batches"
    token_embeddings = torch.squeeze(token_embeddings, dim=1)

    # swap dimensions 0 and 1 so we can loop over tokens
    token_embeddings = token_embeddings.permute(1,0,2)

    # intialized list to store embeddings
    token_vecs_sum = []

    # "token_embeddings" is a [Y x 12 x 768] tensor
    # where Y is the number of tokens in the sentence

    # loop over tokens in sentence
    for token in token_embeddings:

        # "token" is a [12 x 768] tensor

        # sum the vectors from the last four layers
        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs_sum.append(sum_vec)

    return token_vecs_sum

Now we can create contextual embeddings for a set of contexts.

In [17]:
lines = []
with open('sents.txt', 'r') as f:
  lines = f.readlines()

for i in range(len(lines)):
  lines[i] = lines[i].strip()

In [18]:
print(lines)

['еще минувшей весной realme представила в индии смартфоны realme 6 и realme 6 pro, называя их новыми флагманами.', 'для индийского рынка это, безусловно, флагманы.', 'но на фоне продукции топовых мировых брендов для международного рынка новинки предлагают слишком много компромиссов (например, в них используются дисплеи с поддержкой частоты обновления 90 гц, но типа ips, а не amoled).', 'с другой стороны, и цены realme 6/pro — как у индийских флагманов.', 'так что на новинки молодого китайского бренда стоит обратить самое пристальное внимание, и в первую очередь — на самую технически мощную модель, realme 6 pro.', 'о ней сегодня и пойдет наш подробный рассказ.', 'дизайн новинка получила, прямо скажем, простенький, совсем без изюминки и из непремиальных материалов.', 'корпус имеет совсем простую обтекаемую форму, плоской рамки по боковому периметру нет, по сути это единый пластиковый кожух, склеенный со стеклом экрана.', 'пластик глянцевый, скользкий и маркий, бликует на солнце.', 'он и

In [93]:
from collections import OrderedDict
from tqdm import tqdm

context_embeddings = []
context_tokens = []
tokenized_texts = []

for sentence in tqdm(lines):
  tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(sentence, tokenizer)
  tokenized_texts.append(tokenized_text)
  list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)

  # make ordered dictionary to keep track of the position of each word
  tokens = OrderedDict()

  # loop over tokens in sensitive sentence
  for token in tokenized_text[1:-1]:
    # keep track of position of word and whether it occurs multiple times
    if token in tokens:
      tokens[token] += 1
    else:
      tokens[token] = 1

    # compute the position of the current token
    token_indices = [i for i, t in enumerate(tokenized_text) if t == token]
    current_index = token_indices[tokens[token]-1]

    # get the corresponding embedding
    token_vec = list_token_embeddings[current_index]

    # save values
    context_tokens.append(token)
    context_embeddings.append(token_vec)

100%|██████████| 1002/1002 [02:46<00:00,  6.02it/s]


# 3. Результаты

In [96]:
from scipy.spatial.distance import cosine

def get_least_distance(token):
  indices = [i for i, t in enumerate(context_tokens) if t == token]
  token_embeddings = [context_embeddings[i] for i in indices]

  list_of_distances = []

  lines1 = [line for i, line in enumerate(lines) if token in tokenized_texts[i]]

  for sentence_1, embed1 in zip(lines1, token_embeddings):
      for sentence_2, embed2 in zip(lines1, token_embeddings):
          cos_dist = 1 - cosine(embed1, embed2)
          list_of_distances.append([sentence_1, sentence_2, cos_dist])

  distances_df = pd.DataFrame(list_of_distances, columns=['sentence_1', 'sentence_2', 'distance'])

  distances_df = distances_df.sort_values(by='distance')
  if len(distances_df) == 0:
    return None, None, None
  return distances_df['sentence_1'].iloc[0], distances_df['sentence_2'].iloc[0], distances_df['distance'].iloc[0]


In [97]:
words = []
with open('words.txt', 'r') as f:
  words = f.readlines()
words = [word.strip() for word in words]


In [98]:
threshold = 0.4

for word in words:
  s1, s2, d = get_least_distance(word)
  if s1 is None:
    continue
  if d < threshold:
    print(f"Word: {word}, distance: {d}")
    print(s1)
    print(s2)
    print()

Word: стоит, distance: 0.37563368678092957
есть отдельный модуль с телеобъективом (12 мп, f/2,5, pdaf-автофокус), реализующий оптический зум 2× и гибридный 5×. в видоискателе «отбивка» стоит на широком угле и значениях 1×, 2×, 5×. максимальное цифровое приближение возможно до 20×. плавной регулировки зумирования для фото, как для видео, нет, но можно приближать до любого значения щипком пальцев.оптический зум 2×гибридный зум 5×оптический зум 2×гибридный зум 5×камера с пятикратным-то зумированием справляется средне, а про 20× и говорить нечего.
стоит отметить, что в отличие тех же смартфонов huawei/honor, переключение на полное разрешение реализовано максимально удобно, в одно нажатие.16 мп64 мп16 мп64 мп16 мп64 мп.

Word: быстро, distance: 0.3873288333415985
быстро выложили на смазанную маслом нижнюю панель, верхнюю установили в положение 2.
смартфон в меру скользкий и маркий, отпечатками пальцев корпус покрывается не быстро.

Word: производитель, distance: 0.344258576631546
производит

## Выводы

Выше находится вывод для первой тысячи предложений. В файле `examples/lecical_hom.txt` можно посмотреть вывод для 5000 предложений.

Снова результаты не самые лучшие. Очень много неправильно опреденных слов. Попались омографы (стОит / стоИт). Еще в некоторых местах попали слова, которых нет в предложении (это особенности токенайзера BERT, напрмер `производитель` вместо `производительного`). 

Попались грамматические омонимы: `получается`, и несколько других.

Лексические из большого файла: вывод, графики. Кажется, больше нет.